Import data

In [58]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as pyplot
df = pd.read_csv('../iscxids2012-master/iscxids2012-master/data/CSV/TestbedSunJun13Flows.csv')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [45]:
df.head()

,generated,appName,totalSourceBytes,totalDestinationBytes,totalDestinationPackets,totalSourcePackets,sourcePayloadAsBase64,sourcePayloadAsUTF,destinationPayloadAsBase64,destinationPayloadAsUTF,...,sourceTCPFlagsDescription,destinationTCPFlagsDescription,source,protocolName,sourcePort,destination,destinationPort,startDateTime,stopDateTime,Label
0,3/11/2014 17:23,HTTPWeb,128,64,1,2,NaN,NaN,NaN,NaN,...,"F,A","F,A",192.168.1.104,tcp_ip,22441,216.246.64.49,80,6/12/2010 23:58,6/12/2010 23:58,Normal
1,3/11/2014 17:23,HTTPWeb,128,64,1,2,NaN,NaN,NaN,NaN,...,"F,A","F,A",192.168.1.104,tcp_ip,22445,216.246.64.66,80,6/12/2010 23:58,6/12/2010 23:58,Normal
2,3/11/2014 17:23,HTTPWeb,128,64,1,2,NaN,NaN,NaN,NaN,...,"F,A","F,A",192.168.1.104,tcp_ip,22444,216.246.64.66,80,6/12/2010 23:58,6/12/2010 23:58,Normal
3,3/11/2014 17:23,SSH,2938,49570,76,27,NaN,NaN,NaN,NaN,...,"S,R,P,A","S,P,A",192.168.3.115,tcp_ip,3248,192.168.5.122,22,6/12/2010 23:58,6/12/2010 23:58,Normal
4,3/11/2014 17:23,HTTPImageTransfer,644,2315,5,7,NaN,NaN,NaN,NaN,...,"S,P,A","S,P,A",192.168.2.110,tcp_ip,1867,142.166.14.77,80,6/12/2010 23:58,6/12/2010 23:58,Normal


In [46]:
print(df.columns)

Index(['generated', 'appName', 'totalSourceBytes', 'totalDestinationBytes',
       'totalDestinationPackets', 'totalSourcePackets',
       'sourcePayloadAsBase64', 'sourcePayloadAsUTF',
       'destinationPayloadAsBase64', 'destinationPayloadAsUTF', 'direction',
       'sourceTCPFlagsDescription', 'destinationTCPFlagsDescription', 'source',
       'protocolName', 'sourcePort', 'destination', 'destinationPort',
       'startDateTime', 'stopDateTime', 'Label'],
      dtype='object')


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133193 entries, 0 to 133192
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   generated                       133193 non-null  object
 1   appName                         133193 non-null  object
 2   totalSourceBytes                133193 non-null  int64 
 3   totalDestinationBytes           133193 non-null  int64 
 4   totalDestinationPackets         133193 non-null  int64 
 5   totalSourcePackets              133193 non-null  int64 
 6   sourcePayloadAsBase64           70040 non-null   object
 7   sourcePayloadAsUTF              70040 non-null   object
 8   destinationPayloadAsBase64      68042 non-null   object
 9   destinationPayloadAsUTF         68041 non-null   object
 10  direction                       133193 non-null  object
 11  sourceTCPFlagsDescription       94981 non-null   object
 12  destinationTCPFlagsDescription

In [48]:
cols = ['generated', 'appName', 'direction', 'destinationPayloadAsUTF', 'destinationPayloadAsBase64', 'sourcePayloadAsUTF', 'sourcePayloadAsBase64']
df1 = df.drop(cols, axis=1).copy()

In [49]:
df1.head()

,totalSourceBytes,totalDestinationBytes,totalDestinationPackets,totalSourcePackets,sourceTCPFlagsDescription,destinationTCPFlagsDescription,source,protocolName,sourcePort,destination,destinationPort,startDateTime,stopDateTime,Label
0,128,64,1,2,"F,A","F,A",192.168.1.104,tcp_ip,22441,216.246.64.49,80,6/12/2010 23:58,6/12/2010 23:58,Normal
1,128,64,1,2,"F,A","F,A",192.168.1.104,tcp_ip,22445,216.246.64.66,80,6/12/2010 23:58,6/12/2010 23:58,Normal
2,128,64,1,2,"F,A","F,A",192.168.1.104,tcp_ip,22444,216.246.64.66,80,6/12/2010 23:58,6/12/2010 23:58,Normal
3,2938,49570,76,27,"S,R,P,A","S,P,A",192.168.3.115,tcp_ip,3248,192.168.5.122,22,6/12/2010 23:58,6/12/2010 23:58,Normal
4,644,2315,5,7,"S,P,A","S,P,A",192.168.2.110,tcp_ip,1867,142.166.14.77,80,6/12/2010 23:58,6/12/2010 23:58,Normal


In [50]:
df1['startDateTime'] = pd.to_datetime(df1['startDateTime'], format='%m/%d/%Y %H:%M')
df1['stopDateTime'] = pd.to_datetime(df1['stopDateTime'], format='%m/%d/%Y %H:%M')
df1['duration'] = df1['stopDateTime'] - df1['startDateTime']
df1['sessionDuration'] = df1['duration'].apply(lambda x: x.total_seconds())
df1.drop(columns=['duration'], inplace=True)

In [51]:
df1.isnull()

,totalSourceBytes,totalDestinationBytes,totalDestinationPackets,totalSourcePackets,sourceTCPFlagsDescription,destinationTCPFlagsDescription,source,protocolName,sourcePort,destination,destinationPort,startDateTime,stopDateTime,Label,sessionDuration
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133188,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False
133189,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False
133190,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False
133191,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Handling missing values

In [52]:
df1['sourceTCPFlagsDescription'].fillna('None', inplace=True)
df1['destinationTCPFlagsDescription'].fillna('None', inplace=True)

In [53]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133193 entries, 0 to 133192
Data columns (total 15 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   totalSourceBytes                133193 non-null  int64         
 1   totalDestinationBytes           133193 non-null  int64         
 2   totalDestinationPackets         133193 non-null  int64         
 3   totalSourcePackets              133193 non-null  int64         
 4   sourceTCPFlagsDescription       133193 non-null  object        
 5   destinationTCPFlagsDescription  133193 non-null  object        
 6   source                          133193 non-null  object        
 7   protocolName                    133193 non-null  object        
 8   sourcePort                      133193 non-null  int64         
 9   destination                     133193 non-null  object        
 10  destinationPort                 133193 non-null  int64  

In [63]:
import ipaddress
from numpy import int64
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

def ip_to_int(ip_str):
    return int(ipaddress.ip_address(ip_str))

# Load your dataframe `df1`
# df1 = pd.read_csv('path_to_your_data.csv')  # Uncomment and modify according to your actual data

# Check class distribution
print(df1['Label'].value_counts())

# Preparing features and labels
X = df1.drop(['Label'], axis=1)
y = df1['Label'].apply(lambda x: 1 if x == 'Anomaly' else 0)  # Convert label to binary format

# Encoding categorical features before splitting the data
encoder = LabelEncoder()

# Encode categorical features
X['sourceTCPFlagsDescription'] = encoder.fit_transform(X['sourceTCPFlagsDescription'])
X['destinationTCPFlagsDescription'] = encoder.fit_transform(X['destinationTCPFlagsDescription'])
X['protocolName'] = encoder.fit_transform(X['protocolName'])
X['sessionDuration'] = encoder.fit_transform(X['sessionDuration'])
X['startDateTime'] = pd.to_datetime(X['startDateTime']).astype(int64) / 10**9  # Convert to Unix timestamp
X['stopDateTime'] = pd.to_datetime(X['stopDateTime']).astype(int64) / 10**9  # Convert to Unix timestamp

# IP address conversion
X['source'] = X['source'].apply(ip_to_int)
X['destination'] = X['destination'].apply(ip_to_int)

# Check class distribution before splitting
print("Class distribution before split:")
print(y.value_counts())

# Ensure at least one instance of each class before splitting
if y.nunique() < 2:
    raise ValueError("The dataset must have at least two classes.")

# Splitting data into training and test sets with stratified split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Check class distribution after split
print("Class distribution in training set:")
print(y_train.value_counts())
print("Class distribution in test set:")
print(y_test.value_counts())

# Handling class imbalance with SMOTE if the training set has more than one class
if len(y_train.unique()) > 1:
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
else:
    print("Insufficient class diversity in the training set, consider adjusting the split parameters.")
    # Handle this case manually or raise an error
    raise ValueError("Insufficient class diversity in the training set after splitting.")

# Check data types
print(X_train_resampled.dtypes)

# Model training
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train_resampled, y_train_resampled)

# Prediction
y_pred = model.predict(X_test)

# Evaluation
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))



Label
Normal    131107
Attack      2086
Name: count, dtype: int64
Class distribution before split:
Label
0    133193
Name: count, dtype: int64


ValueError: The dataset must have at least two classes.

In [57]:
import ipaddress
from numpy import int64
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder

def ip_to_int(ip_str):
    return int(ipaddress.ip_address(ip_str))

# Simulated dataframe `df1`
# df1 = pd.read_csv('path_to_your_data.csv')  # Uncomment and modify according to your actual data

# Assuming df1 is already defined and loaded with the correct data structure
protocol_mapping = {
    'tcp_ip': 0,
    'udp_ip': 1,
    'icmp_ip': 2, 
}

# Preparing features and labels
X = df1.drop(['Label'], axis=1)
y = df1['Label'].apply(lambda x: 1 if x == 'Anomaly' else 0)  # Convert label to binary format

# Encoding categorical features before splitting the data
encoder = LabelEncoder()

# Encode categorical features
X['sourceTCPFlagsDescription'] = encoder.fit_transform(X['sourceTCPFlagsDescription'])
X['destinationTCPFlagsDescription'] = encoder.fit_transform(X['destinationTCPFlagsDescription'])
X['protocolName'] = encoder.fit_transform(X['protocolName'])
X['sessionDuration'] = encoder.fit_transform(X['sessionDuration'])
X['startDateTime'] = pd.to_datetime(X['startDateTime']).astype(int64) / 10**9  # Convert to Unix timestamp
X['stopDateTime'] = pd.to_datetime(X['stopDateTime']).astype(int64) / 10**9  # Convert to Unix timestamp

# IP address conversion
X['source'] = X['source'].apply(ip_to_int)
X['destination'] = X['destination'].apply(ip_to_int)

# Splitting data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check data types
print(X_train.dtypes)

# Model training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prediction
y_pred = model.predict(X_test)

# Evaluation
print(classification_report(y_test, y_pred))


totalSourceBytes                    int64
totalDestinationBytes               int64
totalDestinationPackets             int64
totalSourcePackets                  int64
sourceTCPFlagsDescription           int32
destinationTCPFlagsDescription      int32
source                              int64
protocolName                        int32
sourcePort                          int64
destination                         int64
destinationPort                     int64
startDateTime                     float64
stopDateTime                      float64
sessionDuration                     int64
dtype: object
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     26639

    accuracy                           1.00     26639
   macro avg       1.00      1.00      1.00     26639
weighted avg       1.00      1.00      1.00     26639



Model Development

In [55]:
import ipaddress
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder

def ip_to_int(ip_str):
    return int(ipaddress.ip_address(ip_str))

protocol_mapping = {
    'tcp_ip': 0,
    'udp_ip': 1,
    'icmp_ip': 2, 
}


encoder = LabelEncoder()
X = df1.drop(['Label'], axis=1)
y = df1['Label'].apply(lambda x: 1 if x == 'Anomaly' else 0)  # Convert label to binary format

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train['sourceTCPFlagsDescription'] = encoder.fit_transform(X_train['sourceTCPFlagsDescription'])
X_train['destinationTCPFlagsDescription'] = encoder.fit_transform(X_train['destinationTCPFlagsDescription'])
X_train['source'] = X_train['source'].apply(ip_to_int)
X_train['protocolName'] =encoder.fit_transform(X_train['protocolName'])
X_train['destination'] = X_train['destination'].apply(ip_to_int)
X_train['sessionDuration'] =encoder.fit_transform(X_train['sessionDuration'])
X_train['startDateTime'] = encoder.fit_transform(X_train['startDateTime'])
X_train['stopDateTime'] = encoder.fit_transform(X_train['stopDateTime'])

X_test['sourceTCPFlagsDescription'] = encoder.fit_transform(X_test['sourceTCPFlagsDescription'])
X_test['destinationTCPFlagsDescription'] = encoder.fit_transform(X_test['destinationTCPFlagsDescription'])
X_test['source'] = X_test['source'].apply(ip_to_int)
X_test['protocolName'] =encoder.fit_transform(X_test['protocolName'])
X_test['destination'] = X_test['destination'].apply(ip_to_int)
X_test['sessionDuration'] =encoder.fit_transform(X_test['sessionDuration'])
X_test['startDateTime'] =encoder.fit_transform(X_test['startDateTime'])
X_test['stopDateTime'] =encoder.fit_transform(X_test['stopDateTime'])
print(X_train.dtypes)
print(X_train['startDateTime'])

model = RandomForestClassifier(n_estimators=100, random_state=42)


model.fit(X_train, y_train)


y_pred = model.predict(X_test)


print(classification_report(y_test, y_pred))


totalSourceBytes                  int64
totalDestinationBytes             int64
totalDestinationPackets           int64
totalSourcePackets                int64
sourceTCPFlagsDescription         int32
destinationTCPFlagsDescription    int32
source                            int64
protocolName                      int32
sourcePort                        int64
destination                       int64
destinationPort                   int64
startDateTime                     int64
stopDateTime                      int64
sessionDuration                   int64
dtype: object
51156      782
11426     1204
47579      787
80511      452
96667      382
          ... 
110268     342
119879     200
103694     361
131932      12
121958     170
Name: startDateTime, Length: 106554, dtype: int64
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     26639

    accuracy                           1.00     26639
   macro avg       1.00      1.00      1.00    

In [64]:
import numpy as np
import pandas as pd
import ipaddress
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Function to convert IP addresses to integers
def ip_to_int(ip_str):
    return int(ipaddress.ip_address(ip_str))

# Dropping unnecessary columns
cols = ['generated', 'appName', 'direction', 'destinationPayloadAsUTF', 'destinationPayloadAsBase64', 'sourcePayloadAsUTF', 'sourcePayloadAsBase64']
df1 = df.drop(cols, axis=1).copy()

# Converting datetime columns
df1['startDateTime'] = pd.to_datetime(df1['startDateTime'], format='%m/%d/%Y %H:%M')
df1['stopDateTime'] = pd.to_datetime(df1['stopDateTime'], format='%m/%d/%Y %H:%M')
df1['duration'] = df1['stopDateTime'] - df1['startDateTime']
df1['sessionDuration'] = df1['duration'].apply(lambda x: x.total_seconds())
df1.drop(columns=['duration'], inplace=True)

# Handling missing values
df1['sourceTCPFlagsDescription'].fillna('None', inplace=True)
df1['destinationTCPFlagsDescription'].fillna('None', inplace=True)

# Converting IP addresses to integers
df1['source'] = df1['source'].apply(ip_to_int)
df1['destination'] = df1['destination'].apply(ip_to_int)

# Preparing features and labels
X = df1.drop(['Label'], axis=1)
y = df1['Label'].apply(lambda x: 1 if x == 'Anomaly' else 0)
print(y) # Convert label to binary format

# Encoding categorical features
encoder = LabelEncoder()
X['sourceTCPFlagsDescription'] = encoder.fit_transform(X['sourceTCPFlagsDescription'])
X['destinationTCPFlagsDescription'] = encoder.fit_transform(X['destinationTCPFlagsDescription'])
X['protocolName'] = encoder.fit_transform(X['protocolName'])

# Converting datetime columns to Unix timestamp
X['startDateTime'] = pd.to_datetime(X['startDateTime']).astype(np.int64) // 10**9
X['stopDateTime'] = pd.to_datetime(X['stopDateTime']).astype(np.int64) // 10**9

# Handling class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Training the RandomForest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predicting on the test set
y_pred = rf_model.predict(X_test)
y_prob = rf_model.predict_proba(X_test)[:, 1]

# Evaluating the model
print(classification_report(y_test, y_pred))
print(f'ROC-AUC Score: {roc_auc_score(y_test, y_prob)}')


ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead